# ZeRO и FSDP: параллелизм, шардирование и конфигурации

## __[DeepSpeed ZeRO](https://www.deepspeed.ai/docs/config-json/):__

Шардирование трёх ключевых состояний модели при обучении с data-parallel:  
- состояния оптимизатора (optimizer states)  
- градиенты (gradients)  
- параметры модели (parameters)  

Это снижает потребление памяти на каждый процесс/устройство без потери вычислительной гранулярности.

__Уровни (stage):__

| Stage | Что шардится | Комментарий |
|-------|-------------|-------------|
| 0 | ничего (традиционный data-parallel, репликация всех состояний) | обычный DDP |
| 1 | optimizer states | градиенты и параметры остаются реплицированными. Экономия памяти по сравнению с 0, небольшой наклад на коммуникации. |
| 2 | optimizer states + gradients | параметры всё ещё реплицированы. Больше экономия, больше наклад. |
| 3 | optimizer states + gradients + parameters | максимальная экономия памяти за счёт полной шардизации. Но самая большая нагрузка на коммуникации. |

__DeepSpeed config:__

```python
zero_optimization = {
    "stage": 0,                                         # 0|1|2|3 — уровень ZeRO, 0 = обычный DDP
    "allgather_partitions": True,                       # собирать параметры после шага (True — дефолт)
    "reduce_scatter": True,                             # использовать reduce_scatter вместо allreduce
    "allgather_bucket_size": 5e8,                       # размер бакета (в элементах, не байтах), дефолт 500М
    "reduce_bucket_size": 5e8,                          # размер бакета для reduce-scatter, дефолт 500М
    "overlap_comm": False,                              # дефолт False; True перекрывает коммуникации с вычислениями
    "contiguous_gradients": True,                       # хранить градиенты в непрерывном блоке памяти
    "offload_optimizer": {                              # offload оптимизатора (актуально при stage >= 2)
        "device": "none",                               # варианты: "none" | "cpu" | "nvme"
        "pin_memory": False
    },
    "offload_param": {                                  # offload параметров (актуально при stage == 3)
        "device": "none",                               # "none" | "cpu" | "nvme"
        "pin_memory": False
    },
    "sub_group_size": 1e9,                              # лимит размера подгруппы при stage3 (байты)
    "stage3_max_live_parameters": 1e9,                  # ограничение активных параметров в памяти
    "stage3_max_reuse_distance": 1e9,                   # макс. расстояние повторного использования параметров
    "stage3_prefetch_bucket_size": 5e8,                 # размер префетч-бакета
    "stage3_param_persistence_threshold": 1e5,          # порог для параметров, не выгружаемых
    "stage3_gather_fp16_weights_on_model_save": True,   # собирать веса при сохранении
}
```
___

## [FSDP](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.TrainingArguments.fsdp) (Fully Sharded Data Parallel)

Метод распределённого обучения, реализованный в PyTorch и интегрированный в библиотеку Accelerate / Transformers. Он позволяет шардировать параметры модели, градиенты и/или optimizer-состояния между устройствами, что значительно снижает расход памяти на GPU.

__FSDP config:__

```python
fsdp_config = {
    "fsdp_sharding_strategy": "FULL_SHARD",             # FULL_SHARD | SHARD_GRAD_OP | NO_SHARD | HYBRID_SHARD | HYBRID_SHARD_ZERO2
    "fsdp_offload_params": False,                       # False = параметры не выгружаются на CPU
    "fsdp_auto_wrap_policy": "TRANSFORMER_BASED_WRAP",  # TRANSFORMER_BASED_WRAP | SIZE_BASED_WRAP | NO_WRAP
    "fsdp_transformer_layer_cls_to_wrap": None,         # список классов слоёв (например ["BertLayer"]); None = автоопределение
    "fsdp_backward_prefetch_policy": "BACKWARD_POST",   # BACKWARD_PRE | BACKWARD_POST | NO_PREFETCH
    "fsdp_forward_prefetch": False,                     # False = не префетчить вперёд
    "fsdp_state_dict_type": "SHARDED_STATE_DICT",       # FULL_STATE_DICT | SHARDED_STATE_DICT | LOCAL_STATE_DICT (рекомендовано SHARDED)
    "fsdp_cpu_ram_efficient_loading": True,             # True = оптимизация загрузки RAM
    "fsdp_min_num_params": 1e8,                         # минимальное кол-во параметров для SIZE_BASED_WRAP (100 млн)
    "fsdp_sync_module_states": True,                    # синхронизация параметров при инициализации
    "fsdp_use_orig_params": True,                       # True = хранить ссылки на оригинальные параметры модели
    "fsdp_activation_checkpointing": False,             # использовать ли checkpointing активаций (экономия памяти)
}

```

[link](https://huggingface.co/docs/accelerate/en/usage_guides/fsdp?utm_source=chatgpt.com)


# Runs

___
### Baseline_1_GPU

In [ ]:
name = 'Baseline_1_GPU'  # 21005 MiB

!CUDA_VISIBLE_DEVICES=1 accelerate launch \
  --num_processes 1 \
  --num_machines 1 \
  --main_process_port 29501 \
  /app/train_distributed.py \
    --mode baseline \
    --bf16 \
    --torch-compile \
    --batch-size 16 \
    --grad-accum 4 \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 17:34:41,692 [INFO] __main__: ============================================================
2025-11-03 17:34:41,692 [INFO] __main__: Запуск обучения в режиме: baseline
2025-11-03 17:34:41,692 [INFO] __main__: ============================================================
2025-11-03 17:34:41,692 [INFO] __main__: World size: 1
2025-11-03 17:34:41,692 [INFO] __main__: CUDA_VISIBLE_DEVICES: 1
2025-11-03 17:34:41,692 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 17:34:41,692 [INFO] __main__: Создание baseline setup (single GPU)
Training samples:   1940063
Validation samples: 5000
2025-11-03 17:35:16,757 [INFO] lib.modeling: Модель: 960,881,664 параметров, dtype=torch.bfloat16, ~1.

___
### 1_GPU_v2

In [ ]:
name = '1_GPU_v2'  # 34353 MiB

!CUDA_VISIBLE_DEVICES=0 accelerate launch \
  --num_processes 1 \
  --num_machines 1 \
  --main_process_port 29502 \
  /app/train_distributed.py \
    --mode baseline \
    --bf16 \
    --torch-compile \
    --batch-size 32 \
    --grad-accum 4 \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 17:38:49,235 [INFO] __main__: ============================================================
2025-11-03 17:38:49,235 [INFO] __main__: Запуск обучения в режиме: baseline
2025-11-03 17:38:49,235 [INFO] __main__: ============================================================
2025-11-03 17:38:49,235 [INFO] __main__: World size: 1
2025-11-03 17:38:49,235 [INFO] __main__: CUDA_VISIBLE_DEVICES: 0
2025-11-03 17:38:49,235 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 17:38:49,236 [INFO] __main__: Создание baseline setup (single GPU)
Training samples:   1940063
Validation samples: 5000
2025-11-03 17:39:25,437 [INFO] lib.modeling: Модель: 960,881,664 параметров, dtype=torch.bfloat16, ~1.

___
### DeepSpeed_1

In [ ]:
name = 'DeepSpeed_1'  # 35523 MiB

!CUDA_VISIBLE_DEVICES=2,3 accelerate launch \
  --num_processes 2 \
  --num_machines 1 \
  --main_process_port 29503 \
  /app/train_distributed.py \
    --mode deepspeed \
    --deepspeed-stage 1 \
    --reduce-bucket-size 500000000 \
    --allgather-bucket-size 500000000 \
    --overlap-comm \
    --bf16 \
    --batch-size 16 \
    --grad-accum 4 \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 17:41:17,720 [INFO] __main__: ============================================================
2025-11-03 17:41:17,720 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-03 17:41:17,720 [INFO] __main__: ============================================================
2025-11-03 17:41:17,720 [INFO] __main__: World size: 2
2025-11-03 17:41:17,720 [INFO] __main__: CUDA_VISIBLE_DEVICES: 2,3
2025-11-03 17:41:17,720 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 17:41:17,721 [INFO] __main__: Создание DeepSpeed setup (stage 1)
2025-11-03 17:41:17,721 [INFO] __

___
### DeepSpeed_2

In [ ]:
name = 'DeepSpeed_2'  # 33135 MiB

!CUDA_VISIBLE_DEVICES=0,1 accelerate launch \
  --num_processes 2 \
  --num_machines 1 \
  --main_process_port 29504 \
  /app/train_distributed.py \
    --mode deepspeed \
    --deepspeed-stage 2 \
    --reduce-bucket-size 500000000 \
    --allgather-bucket-size 500000000 \
    --overlap-comm \
    --bf16 \
    --batch-size 16 \
    --grad-accum 4 \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 18:12:33,701 [INFO] __main__: ============================================================
2025-11-03 18:12:33,701 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-03 18:12:33,701 [INFO] __main__: ============================================================
2025-11-03 18:12:33,701 [INFO] __main__: World size: 2
2025-11-03 18:12:33,701 [INFO] __main__: CUDA_VISIBLE_DEVICES: 0,1
2025-11-03 18:12:33,701 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 18:12:33,701 [INFO] __main__: Создание DeepSpeed setup (stage 2)
2025-11-03 18:12:33,701 [INFO] __

___
### DeepSpeed_3

In [ ]:
name = 'DeepSpeed_3'  # 36829 MiB

!CUDA_VISIBLE_DEVICES=2,3 accelerate launch \
  --num_processes 2 \
  --num_machines 1 \
  --main_process_port 29505 \
  /app/train_distributed.py \
    --mode deepspeed \
    --deepspeed-stage 3 \
    --reduce-bucket-size 500000000 \
    --allgather-bucket-size 500000000 \
    --overlap-comm \
    --stage3-prefetch-bucket-size 500000000 \
    --stage3-param-persistence-threshold 100000 \
    --bf16 \
    --batch-size 16 \
    --grad-accum 4 \
    --no-generation \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 18:15:35,846 [INFO] __main__: ============================================================
2025-11-03 18:15:35,846 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-03 18:15:35,846 [INFO] __main__: ============================================================
2025-11-03 18:15:35,846 [INFO] __main__: World size: 2
2025-11-03 18:15:35,846 [INFO] __main__: CUDA_VISIBLE_DEVICES: 2,3
2025-11-03 18:15:35,846 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 18:15:35,846 [INFO] __main__: Создание DeepSpeed setup (stage 3)
2025-11-03 18:15:35,846 [INFO] __

___
### FSDP_full_shard

In [ ]:
name = 'FSDP_full_shard'  # 30117 MiB
# FULL_SHARD | SHARD_GRAD_OP | NO_SHARD | HYBRID_SHARD | HYBRID_SHARD_ZERO2
# "full_shard", "shard_grad_op", "no_shard", "hybrid_shard"
    # --fsdp-cpu-offload \ выгрузка параметров на CPU (Уменьшает память GPU, но замедляет обучение)
    # --fsdp-activation-checkpointing \  Экономит память, но замедляет (~30% накладных) (Пересчитывает активации вместо сохранения)

!CUDA_VISIBLE_DEVICES=0,1 accelerate launch \
  --num_processes 2 \
  --num_machines 1 \
  --main_process_port 29501 \
  /app/train_distributed.py \
    --mode fsdp \
    --fsdp-sharding-strategy full_shard \
    --fsdp-backward-prefetch BACKWARD_POST \
    --fsdp-forward-prefetch \
    --fsdp-state-dict-type FULL_STATE_DICT \
    --fsdp-transformer-layers Qwen3DecoderLayer \
    --bf16 \
    --batch-size 16 \
    --grad-accum 4 \
    --no-generation \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 19:17:16,464 [INFO] __main__: ============================================================
2025-11-03 19:17:16,464 [INFO] __main__: Запуск обучения в режиме: fsdp
2025-11-03 19:17:16,464 [INFO] __main__: ============================================================
2025-11-03 19:17:16,464 [INFO] __main__: World size: 2
2025-11-03 19:17:16,465 [INFO] __main__: CUDA_VISIBLE_DEVICES: 0,1
2025-11-03 19:17:16,465 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 19:17:16,465 [INFO] __main__: Создание FSDP setup
2025-11-03 19:17:16,465 [INFO] __main__:   sharding_s

___
### FSDP_full_shard_v2

In [ ]:
name = 'FSDP_full_shard_v2'  # 30117 MiB

!CUDA_VISIBLE_DEVICES=2,3 accelerate launch \
  --num_processes 2 \
  --num_machines 1 \
  --main_process_port 29502 \
  /app/train_distributed.py \
    --mode fsdp \
    --fsdp-sharding-strategy full_shard \
    --fsdp-backward-prefetch BACKWARD_POST \
    --fsdp-forward-prefetch \
    --fsdp-state-dict-type FULL_STATE_DICT \
    --fsdp-transformer-layers Qwen3DecoderLayer \
    --bf16 \
    --batch-size 16 \
    --grad-accum 2 \
    --no-generation \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 19:37:23,097 [INFO] __main__: ============================================================
2025-11-03 19:37:23,097 [INFO] __main__: Запуск обучения в режиме: fsdp
2025-11-03 19:37:23,097 [INFO] __main__: ============================================================
2025-11-03 19:37:23,190 [INFO] __main__: ============================================================
2025-11-03 19:37:23,190 [INFO] __main__: Запуск обучения в режиме: fsdp
2025-11-03 19:37:23,190 [INFO] __main__: ============================================================
2025-11-03 19:37:23,190 [INFO] __

___
### FSDP_full_shard_v3

In [ ]:
name = 'FSDP_full_shard_v3'  # 20817 MiB

!CUDA_VISIBLE_DEVICES=0,1 accelerate launch \
  --num_processes 2 \
  --num_machines 1 \
  --main_process_port 29501 \
  /app/train_distributed.py \
    --mode fsdp \
    --fsdp-sharding-strategy full_shard \
    --fsdp-backward-prefetch BACKWARD_POST \
    --fsdp-forward-prefetch \
    --fsdp-state-dict-type FULL_STATE_DICT \
    --fsdp-transformer-layers Qwen3DecoderLayer \
    --bf16 \
    --batch-size 8 \
    --grad-accum 4 \
    --no-generation \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 19:51:37,530 [INFO] __main__: ============================================================
2025-11-03 19:51:37,530 [INFO] __main__: Запуск обучения в режиме: fsdp
2025-11-03 19:51:37,530 [INFO] __main__: ============================================================
2025-11-03 19:51:37,530 [INFO] __main__: World size: 2
2025-11-03 19:51:37,530 [INFO] __main__: CUDA_VISIBLE_DEVICES: 0,1
2025-11-03 19:51:37,530 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 19:51:37,530 [INFO] __main__: Создание FSDP setup
2025-11-03 19:51:37,530 [INFO] __main__:   sharding_s

___
### FSDP_full_shard_v4

In [ ]:
name = 'FSDP_full_shard_v4'  # 61781 MiB

!CUDA_VISIBLE_DEVICES=2,3 accelerate launch \
  --num_processes 2 \
  --num_machines 1 \
  --main_process_port 29502 \
  /app/train_distributed.py \
    --mode fsdp \
    --fsdp-sharding-strategy full_shard \
    --fsdp-backward-prefetch BACKWARD_POST \
    --fsdp-forward-prefetch \
    --fsdp-state-dict-type FULL_STATE_DICT \
    --fsdp-transformer-layers Qwen3DecoderLayer \
    --fsdp-cpu-offload \
    --fsdp-activation-checkpointing \
    --bf16 \
    --batch-size 128 \
    --grad-accum 1 \
    --no-generation \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-03 20:30:35,497 [INFO] __main__: ============================================================
2025-11-03 20:30:35,497 [INFO] __main__: Запуск обучения в режиме: fsdp
2025-11-03 20:30:35,497 [INFO] __main__: ============================================================
2025-11-03 20:30:35,804 [INFO] __main__: ============================================================
2025-11-03 20:30:35,805 [INFO] __main__: Запуск обучения в режиме: fsdp
2025-11-03 20:30:35,805 [INFO] __main__: ============================================================
2025-11-03 20:30:35,805 [INFO] __

___
### FSDP_full_shard_v5

In [2]:
name = 'FSDP_full_shard_v5'  # 14379 MiB

!CUDA_VISIBLE_DEVICES=0,1,2,3 accelerate launch \
  --num_processes 4 \
  --num_machines 1 \
  --main_process_port 29501 \
  /app/train_distributed.py \
    --mode fsdp \
    --fsdp-sharding-strategy full_shard \
    --fsdp-backward-prefetch BACKWARD_POST \
    --fsdp-forward-prefetch \
    --fsdp-state-dict-type FULL_STATE_DICT \
    --fsdp-transformer-layers Qwen3DecoderLayer \
    --bf16 \
    --batch-size 16 \
    --grad-accum 4 \
    --no-generation \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-04 11:46:58,474 [INFO] __main__: ============================================================
2025-11-04 11:46:58,474 [INFO] __main__: Запуск обучения в режиме: fsdp
2025-11-04 11:46:58,474 [INFO] __main__: ============================================================
2025-11-04 11:46:58,475 [INFO] __main__: World size: 4
2025-11-04 11:46:58,475 [INFO] __main__: CUDA_VISIBLE_DEVICES: 0,1,2,3
2025-11-04 11:46:58,475 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-04 11:46:58,477 [INFO] __main__: Создание FSDP setup
2025-11-04 11:46:58,477 [INFO] __main__:   shardi

___
### FSDP_full_shard_v6

In [4]:
name = 'FSDP_full_shard_v6'  # 11225 MiB

!CUDA_VISIBLE_DEVICES=0,1,2,3 accelerate launch \
  --num_processes 4 \
  --num_machines 1 \
  --main_process_port 29501 \
  /app/train_distributed.py \
    --mode fsdp \
    --fsdp-sharding-strategy full_shard \
    --fsdp-backward-prefetch BACKWARD_POST \
    --fsdp-forward-prefetch \
    --fsdp-state-dict-type FULL_STATE_DICT \
    --fsdp-transformer-layers Qwen3DecoderLayer \
    --bf16 \
    --batch-size 8 \
    --grad-accum 2 \
    --no-generation \
    --run-name {name} \
  2>&1 | tee /app/data/logs/{name}.log

The following values were not passed to `accelerate launch` and had defaults used instead:
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-04 12:29:36,010 [INFO] __main__: ============================================================
2025-11-04 12:29:36,010 [INFO] __main__: Запуск обучения в режиме: fsdp
2025-11-04 12:29:36,010 [INFO] __main__: ============================================================
2025-11-04 12:29:36,014 [INFO] __main__: ============================================================
2025-11-04 12:29:36,014 [INFO] __main__: Запуск обучения в режиме: fsdp
2025-11-04 12:29:36,014 [INFO] __main__: ============================================================
2025-11-04 12:29:36,044 [INFO] __

___
# Отчёт

In [3]:
from lib.utils.stash import parse_logs
import pandas as pd

log_path = "/app/data/logs" 
df = parse_logs(log_path)
pd.set_option('display.max_columns', None)
display(df[['File name', 'CUDA visible devices', 'World size', 'Parallel mode', 'Precision', 'Per-device batch size', 'Gradient accumulation', 'Effective batch size', 'Iter/s', 'Samples/s', 'Total samples seen', 'Steps detected', 'Train runtime', 'Eval loss']])


,File name,CUDA visible devices,World size,Parallel mode,Precision,Per-device batch size,Gradient accumulation,Effective batch size,Iter/s,Samples/s,Total samples seen,Steps detected,Train runtime,Eval loss
0,1_GPU_v2.log,0,1,None,None,NaN,1,NaN,1.90,NaN,NaN,947,1800.49,3.94
1,Baseline_1_GPU.log,1,1,None,None,NaN,1,NaN,1.01,NaN,NaN,1782,1800.48,3.78
2,DeepSpeed_1.log,"2, 3",2,DeepSpeed (ZeRO-1),None,NaN,1,NaN,2.05,NaN,NaN,878,1801.23,4.83
3,DeepSpeed_2.log,"0, 1",2,DeepSpeed (ZeRO-2),None,NaN,1,NaN,3.16,NaN,NaN,570,1801.85,6.08
4,DeepSpeed_3.log,"2, 3",2,DeepSpeed (ZeRO-3),None,NaN,1,NaN,2.60,NaN,NaN,692,1801.25,NaN
5,FSDP_full_shard.log,"0, 1",2,FSDP full_shard auto_wrap,bf16,16.0,4,128.0,2.75,46.55,83840.0,655,1802.24,NaN
6,FSDP_full_shard_v2.log,"2, 3",2,FSDP full_shard auto_wrap,bf16,16.0,2,64.0,1.49,42.95,77312.0,1208,1800.45,NaN
7,FSDP_full_shard_v3.log,"0, 1",2,FSDP full_shard auto_wrap,bf16,8.0,4,64.0,2.26,28.32,51008.0,797,1801.57,5.25
8,FSDP_full_shard_v4.log,"2, 3",2,FSDP full_shard auto_wrap offload,bf16,128.0,1,256.0,8.20,31.22,56320.0,220,1804.72,6.85
9,FSDP_full_shard_v5.log,"0, 1, 2, 3",4,FSDP full_shard auto_wrap,bf16,16.0,4,256.0,3.03,84.49,152064.0,594,1802.52,4.95


Потребление памяти на GPU:
- 21005 MiB — Baseline_1_GPU
- 34353 MiB — 1_GPU_v2
- 35523 MiB — DeepSpeed_1
- 33135 MiB — DeepSpeed_2
- 36829 MiB — DeepSpeed_3
- 30117 MiB — FSDP_full_shard
- 30117 MiB — FSDP_full_shard_v2
- 20817 MiB — FSDP_full_shard_v3
- 61781 MiB — FSDP_full_shard_v4
___
Вот, что я так и не смог понять:

- Почему на 1 GPU у нас моделька занимает меньше места, чем она же, разбитая на 2 GPU посредством DeepSpeed или FSDP (но тут чуть получше). Вероятно дело тут в аллоцированных буферах, я немного с ними экспериментировал, и это действительно снижает потребление памяти. Однако чтобы моделька в DeepSpeed на 2 GPU занимала меньше места чем на 1 мне добиться так и не удалось. Тут я уже начал подозревать, что делаю что-то вообще не так как нужно и у меня моделька просто тупо копируется между девайсами или ещё что, и меня посетила мысль просто закинуть обучение на 4 GPU, точно то же самое что и в FSDP_full_shard, но на 4. 30117 → 14379 MiB на карточку, оно работает! Правда скорость стала ещё медленнее.

- Почему так замедляется обучение.. Казалось бы, у нас под капотом должен работать AFAB, но скорость обучения всё равно падает. Моделька на 2 GPU успевает увидеть меньше сэмплов, чем при Baseline (FSDP_full_shard / FSDP_full_shard_v2 / Baseline :: 83840 / 77312 / 115200). Но успевает увидеть больше сэмплов на 4 GPU (да простит меня Devbox 11). Вероятно тут дело в коммуникациях, я проверил `nvidia-smi topo -m`:

||**GPU0**|**GPU1**|**GPU2**|**GPU3**|**CPU Affinity**|**NUMA Affinity**|**GPU NUMA ID**|
|---|---|---|---|---|---|---|---|
|**GPU0**|X|SYS|SYS|SYS|0–15, 32–47|0|N/A|
|**GPU1**|SYS|X|SYS|SYS|0–15, 32–47|0|N/A|
|**GPU2**|SYS|SYS|X|SYS|16–31, 48–63|1|N/A|
|**GPU3**|SYS|SYS|SYS|X|16–31, 48–63|1|N/A|

- NVLINKов нет, общение через CPU, вероятно ботлнек тут именно коммуникации, и красивая картинка про AFAB (если там под капотом вообще он) работает не так, тонкие чёрные граници между микробатчами превращаются в больщущие (по меркам GPU) задержки при передаче данных между карточками (кстати, в картинке есть расхождения с тем, что у нас на слайде AFAB в презе, картинка с [medium](https://medium.com/better-programming/parallel-and-distributed-training-in-deep-learning-for-beginners-part-1-introduction-612a4534a117)):

![](/app/data/imgs/AFAB.png)

- Дальше, собственно микробатчи. У нас есть world_size - количество GPU, потом gradient accumulation и batch size. И вообще DeepSpeed нас вынуждает делать следующее: `kwargs.setdefault("train_micro_batch_size_per_gpu", training_config["per_device_train_batch_size"])` иначе громко ругается. То есть он принимает свой параметр `micro_batch_size`, который всегда равен просто нашему батчу. Вот тут и вопрос - в какой момент вообще возникают микробатчи и из чего (и как понять что там под капотом у этого DeepSpeed, Tensor parallel или Pipeline parallel с какой-то вариацией AFAB)? Просто по идее обучение то должно всё равно ускоряться (хоть чуть-чуть), неужели всё дело в коммуникациях через SYS, не просто же так у нас там есть `overlap_comm`?

- Также я когда запускал эксперимент `FSDP_full_shard_v4` с огромным батчом, то увидел мощный (порядка 2.5-3 секунд) простой обеих GPU. Они начинают синхронно работать, потом синхронно тупить. Если это из-за коммуникаций с большим батчом и там AFAB, то они должны по идее работать по-очереди с некоторым перекрытием. Снова странно.

___
Вопросы из задания:

- DeepSpeed: Проверьте, получается ли увеличить batch_size за счет освобождения памяти при шардировании. Если да, то насколько эффективнее стало обучение?
    - Получается, но только на большем чем 2 количестве GPU (и чем больше GPU тем медленее обучение)
- Зафиксируйте для отчёта по ДЗ ...
    - Всё в табличке выше и в [wandb](https://wandb.ai/ksorzz/llm_hw2-aylesnov?nw=nwuserksorzz) (добавлял логи постепенно, поэтому кое-где `None`)
- Сравните скорость, использование памяти. Наблюдается ли линейный scaling скорости обучения?
    - `1 GPU (GBS = 64):  115200 / 115200 = 1    (Baseline, eval_loss: 3.78)`
    - `2 GPU (GBS = 128): 112384 / 115200 = 0.98 (DeepSpeed_1, eval_loss: 4.83)`
    - `2 GPU (GBS = 128): 83840  / 115200 = 0.73 (FSDP_full_shard, eval_loss: 5.1)`
    - `4 GPU (GBS = 256): 152064 / 115200 = 1.32 (FSDP_full_shard_v5, eval_loss: 4.95)`
    - В вопросе r < 2 или r > 2, предполагается видимо, что всё таки должен быть почти x2 буст по скорости либо же там ошибка и вопрос должен был стоять как r < 1 или r > 1. Собственно r=0.98 на 2 GPU с stage 1 DeepSpeed - лучшее, чего удалось добиться (хотя сходилось оно хуже, но это из-за global batch size судя по всему, посколько в сингл оптимальным является 64 для этой модельки и этого датасета). r у меня на 2 GPU получилось __меньше 1__, что либо моя ошибка либо это ботлнек коммуникаций между картами.
- Сделайте вывод, какой подход эффективнее для вашей модели:
    - Single-GPU

В целом какие-то такие мысли по поводу задания, вообще всё непонятно, просто удалось увидеть ощутимое снижение GPU utilization (но только на 4 картах).
___